# Task 4. Algorithms for unconstrained nonlinear optimization. Stochastic and metaheuristic algorithms

## Goal
The use of stochastic and metaheuristic algorithms (`Simulated Annealing`, `Differential Evolution`, `Particle Swarm Optimization`) in the tasks of unconstrained nonlinear optimization and the experimental comparison of them with `Nelder-Mead` and `Levenberg-Marquardt` algorithms

In [1]:
import random
from math import sin, cos, sqrt, atan2, radians

import pandas as pd
import numpy as np

from tqdm import tqdm

import scipy
from scipy import optimize, spatial
from scipy.spatial.distance import pdist, squareform

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

## I Generate the noisy data $(x_k, y_k)$, where $k = 0, \dots , 1000$, according to the rule:


$
y_k = 
    \begin{cases}
        −100 + \delta_k, f(x_k) < −100, \newline
        f(x_k) + \delta_k, −100 ≤ f(x_k) ≤ 100, \newline
        100 + \delta_k, f(x_k) > 100 \\
    \end{cases} \qquad
    x_k=\dfrac{3k}{1000},\qquad
    f(x)= \dfrac{1}{x^2+3x+2}
$

where $\delta_k \sim (0,1)$ are values of a random variable with standard normal
distribution. Approximate the data by the following linear and rational function

In [24]:
y_k = []
x_k = []

for k in range(0, 1000):
    b_k = np.random.normal(0, 1)

    x_k.append(3 * k / (1000))

    fx = 1 / (x_k[k] ** 2 - 3 * x_k[k] + 2)

    if fx < -100:
        y_k.append(-100 + b_k)
    elif abs(fx) <= 100:
        y_k.append(fx + b_k)
    else:
        y_k.append(100 + b_k)

x_k = np.array(x_k)
x_k = np.array(x_k)
print(len(x_k))
print(len(y_k))

1000
1000


In [25]:
px.line(x=x_k, y=y_k)

In [26]:
def rational_approx(x, a, b, c, d):
  return (a * x + b) / (x ** 2 + c * x + d)

by means of least squares through the numerical minimization of the following function:
\begin{align*}
D(a, b, c, d) = \sum^{1000}_{k=0} \left(F(x_k, a, b, c ,d) - y_k\right)^2
\end{align*}

In [27]:
# Means of least squares
def D(params, yk, x):
  a, b, c, d = params

  return sum((rational_approx(x, a, b, c, d) - yk) ** 2)

To solve the minimization problem, use `Nelder-Mead algorithm`,`LevenbergMarquardt algorithm` and at least two of the methods among `Simulated Annealing`, `Differential Evolution` and `Particle Swarm Optimization`. If necessary, set the initial approximations and other parameters of the methods. Use $𝜀 = 0.001$ as the precision; at most $1000$ iterations **are allowed**. Visualize the data and the approximants obtained in a single plot. Analyze and compare the results obtained (in terms of number of iterations, precision, number of function evaluations, etc.).

In [28]:
eps = 1e-3

nelderMead = scipy.optimize.minimize(D, [0.1, 0.1, 0.1, 0.1], args=(y_k, x_k), method='Nelder-Mead', tol=eps)
print(f'Nelder-Mead method: {nelderMead.x} function evaluations: {nelderMead.nfev} iterations: {nelderMead.nit}')

Nelder-Mead method: [-1.00048425  1.00098774 -2.00092557  1.00094156] function evaluations: 553 iterations: 327


In [29]:
# Levenberg-Marquardt
def reseduals(params, yk, x): 
  a, b, c, d = params
  res = np.zeros(len(x))
  for i in range(len(x)):
    res[i] = (a * x[i] + b) / (x[i] ** 2 + c * x[i] + d) - yk[i]
  return res

lm = scipy.optimize.least_squares(reseduals, [0.1, 0.1, 0.1, 0.1], args=(y_k, x_k), method='lm', xtol=eps)

print(f'Levenberg-Marquardt method: {lm.x} function evaluations: {lm.nfev}')

Levenberg-Marquardt method: [-1.00135376  1.00183263 -2.0008287   1.0008447 ] function evaluations: 189


In [30]:
# Simulated Annealing
r_min, r_max = -5, 5

annealing = scipy.optimize.dual_annealing(D, ((r_min, r_max), (r_min, r_max), (r_min, r_max), (r_min, r_max)), args=(y_k, x_k))

print(f'Simulated Annealing: {annealing.x} function evaluations: {annealing.nfev} iterations: {annealing.nit}')

Simulated Annealing: [-1.00035494  1.00085833 -2.0009257   1.00094168] function evaluations: 8751 iterations: 1000


In [31]:
# Differential Evolution

diff_evol = scipy.optimize.differential_evolution(D, ((r_min, r_max), (r_min, r_max), (r_min, r_max), (r_min, r_max)), args=(y_k, x_k), maxiter=1000)

print(f'Differential Evolution: {diff_evol.x} function evaluations: {diff_evol.nfev} iterations: {diff_evol.nit}')

Differential Evolution: [-1.00038013  1.00088335 -2.00092547  1.00094146] function evaluations: 1170 iterations: 5


In [32]:
nelderMead_y = []
for k in range(1000):
  nelderMead_y.append((nelderMead.x[0] * x_k[k] + nelderMead.x[1]) / (x_k[k] ** 2 + nelderMead.x[2] * x_k[k] + nelderMead.x[3]))
    
lm_y = []
for k in range(1000):
  lm_y.append((lm.x[0] * x_k[k] + lm.x[1]) / (x_k[k] ** 2 + lm.x[2] * x_k[k] + lm.x[3]))

annealing_y = []
for k in range(1000):
  annealing_y.append((annealing.x[0] * x_k[k] + annealing.x[1]) / (x_k[k] ** 2 + annealing.x[2] * x_k[k] + annealing.x[3]))  

diff_evol_y = []
for k in range(1000):
  diff_evol_y.append((diff_evol.x[0] * x_k[k] + diff_evol.x[1]) / (x_k[k] ** 2 + diff_evol.x[2] * x_k[k] + diff_evol.x[3]))  

fig = go.Figure()
fig = px.scatter(x=x_k, y=y_k)

fig.add_trace(
    go.Scatter(
        x=x_k,
        y=nelderMead_y,
        name="Nelder-Mead"
    )
)


fig.add_trace(
    go.Scatter(
        x=x_k,
        y=lm_y,
        name="Levenberg-Marquardt"
    )
)

fig.add_trace(
    go.Scatter(
        x=x_k,
        y=annealing_y,
        name="Simulated Annealing"
    )
)

fig.add_trace(
    go.Scatter(
        x=x_k,
        y=diff_evol_y,
        name="Differential Evolution"
    )
)

fig.show()

In [33]:
def f_x(x):
  return 1 / (x**2 - 3 * x + 2)
# data generation

x_k = []
y_k = []

for k in range(1001):
  x_k.append(3 * k / 1000)
  f = f_x(x_k[k])
  if(f < -100):
    y_k.append(-100 + np.random.normal(0, 1))
  elif (f >= -100 and f <= 100):
    y_k.append(f + np.random.normal(0, 1))
  else:
    y_k.append(100 + np.random.normal(0, 1))
xk = np.array(x_k)
yk = np.array(y_k)

In [34]:
px.line(x=x_k, y=y_k)

## II. Choose at least 15 cities in the world having land transport connections between them. Calculate the distance matrix for them and then apply the Simulated Annealing method to solve the corresponding Travelling Salesman Problem. Visualize the results at the first and the last iteration. If necessary, use the city dataset from https://people.sc.fsu.edu/~jburkardt/datasets/cities/cities.html

In [2]:
us_cities = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/us-cities-top-1k.csv")
us_cities["city_state"] = us_cities["City"] + ", " + us_cities["State"]
us_cities = us_cities[:25]
us_cities

,City,State,Population,lat,lon,city_state
0,Marysville,Washington,63269,48.051764,-122.177082,"Marysville, Washington"
1,Perris,California,72326,33.782519,-117.228648,"Perris, California"
2,Cleveland,Ohio,390113,41.499320,-81.694361,"Cleveland, Ohio"
3,Worcester,Massachusetts,182544,42.262593,-71.802293,"Worcester, Massachusetts"
4,Columbia,South Carolina,133358,34.000710,-81.034814,"Columbia, South Carolina"
5,Waterbury,Connecticut,109676,41.558153,-73.051496,"Waterbury, Connecticut"
6,Eagan,Minnesota,65453,44.804132,-93.166886,"Eagan, Minnesota"
7,Southfield,Michigan,73006,42.473369,-83.221873,"Southfield, Michigan"
8,Lafayette,Louisiana,124276,30.224090,-92.019843,"Lafayette, Louisiana"
9,Boise City,Idaho,214237,43.618710,-116.214607,"Boise City, Idaho"


In [3]:
def dist(x, y):
    """Function to compute the distance between two points x, y"""

    lat1 = radians(x[0])
    lon1 = radians(x[1])
    lat2 = radians(y[0])
    lon2 = radians(y[1])

    R = 6373.0

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return round(distance, 4)

In [4]:
distances = pdist(us_cities[["lat", "lon"]], metric=dist)

points = [f"point_{i}" for i in range(1, len(us_cities) + 1)]

# result = pd.DataFrame(squareform(distances), columns=us_cities.city_state, index=us_cities.city_state)
result = pd.DataFrame(
    squareform(distances),
    columns=us_cities.city_state,
    index=us_cities.city_state
)
result = result.replace(0, np.nan)
display(result)

city_state,"Marysville, Washington","Perris, California","Cleveland, Ohio","Worcester, Massachusetts","Columbia, South Carolina","Waterbury, Connecticut","Eagan, Minnesota","Southfield, Michigan","Lafayette, Louisiana","Boise City, Idaho",...,"Salem, Massachusetts","Aurora, Illinois","Leesburg, Virginia","Doral, Florida","Westminster, California","Lubbock, Texas","Overland Park, Kansas","Jackson, Mississippi","Gastonia, North Carolina","Daytona Beach, Florida"
city_state,,,,,,,,,,,,,,,,,,,,,
"Marysville, Washington",NaN,1639.8539,3238.8770,3931.2505,3733.6220,3875.6082,2239.5579,3077.8446,3236.3625,675.3522,...,3984.5057,2733.3452,3673.4954,4386.2879,1628.3960,2335.5322,2420.1024,3199.2202,3638.8626,4076.8424
"Perris, California",1639.8539,NaN,3220.4837,4040.5697,3324.1689,3939.8855,2393.2720,3104.6332,2404.3261,1097.5878,...,4113.9339,2677.0167,3569.3637,3646.8284,70.8530,1421.8225,2094.9750,2519.8134,3288.8880,3453.4947
"Cleveland, Ohio",3238.8770,3220.4837,NaN,823.1202,836.0756,719.4141,999.7935,166.3910,1559.2396,2817.6268,...,898.9579,551.4830,439.2990,1748.3952,3285.6401,1977.8849,1135.5649,1270.4269,695.1703,1368.1922
"Worcester, Massachusetts",3931.2505,4040.5697,823.1202,NaN,1221.9835,129.7354,1740.6312,938.0473,2244.7143,3575.9901,...,79.6937,1364.1108,598.6315,1988.9767,4105.1131,2790.7450,1958.6397,1961.7403,1125.1104,1671.0535
"Columbia, South Carolina",3733.6220,3324.1689,836.0756,1221.9835,NaN,1094.0320,1587.4095,961.4876,1116.1352,3203.2335,...,1294.9640,1073.3597,647.8370,912.3371,3394.3221,1921.9332,1337.5004,872.5485,140.9945,532.7805
"Waterbury, Connecticut",3875.6082,3939.8855,719.4141,129.7354,1094.0320,NaN,1666.3626,846.0944,2115.9119,3502.1581,...,207.6456,1267.3653,469.1041,1874.7330,4005.0523,2675.1788,1852.0319,1833.4750,995.7257,1550.2809
"Eagan, Minnesota",2239.5579,2393.2720,999.7935,1740.6312,1587.4095,1666.3626,NaN,840.8295,1624.8430,1836.0578,...,1804.1338,518.1281,1434.0962,2403.7891,2451.3962,1454.1062,659.3980,1414.6749,1469.1189,2037.3352
"Southfield, Michigan",3077.8446,3104.6332,166.3910,938.0473,961.4876,846.0944,840.8295,NaN,1571.9242,2667.2358,...,1009.9309,427.9768,605.1597,1870.7884,3168.6601,1902.7237,1039.1659,1287.0760,821.1609,1488.3051
"Lafayette, Louisiana",3236.3625,2404.3261,1559.2396,2244.7143,1116.1352,2115.9119,1624.8430,1571.9242,NaN,2601.2623,...,2323.5145,1325.3263,1647.8653,1244.8397,2474.9030,1000.3042,1003.8044,289.2859,1157.0291,1067.8568


In [5]:
# Map with our countries

fig = px.line_mapbox(
    us_cities, 
    lat="lat", 
    lon="lon", 
    color="City", 
    zoom=3, 
    height=500
)

fig.update_layout(
    mapbox_style="stamen-terrain", 
    mapbox_zoom=3, 
    mapbox_center_lat=41,
    margin={"r":0, "t":0, "l":0, "b":0}
)

fig.show()

In [6]:
# Random routes
fig = go.Figure()

fig = px.line_mapbox(us_cities, lat="lat", lon="lon", color="City", zoom=3, height=500)

for num in range(0, len(us_cities) - 1):
    fig.add_trace(go.Scattermapbox(
        mode = "markers+lines",

        lon = [us_cities.iloc[num]['lon'], us_cities.iloc[num + 1]['lon']],
        lat = [us_cities.iloc[num]['lat'], us_cities.iloc[num + 1]['lat']],
        marker = {'size': 10}, name=us_cities.iloc[num]['City']))

fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=3, mapbox_center_lat = 41,
    margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

# Последовательная реализация
# Создадим произвольный начальный путь, 
- содержащий все вершины по одному разу и возвращающийся в начальную позицию. 
- Затем случайным образом будем менять местами два города и сравнивать длины старого и нового путей. Если новый путь оказался короче, сохраняем его. Если нет, то наращиваем счетчик. Когда счетчик примет заранее заданное значение, останавливаем алгоритм, последний найденный таким образом путь будет считаться наилучшим.

In [7]:
def path_draw(optimal_path):
    fig = go.Figure()

    index = 0

    temp_city_first = pd.DataFrame()
    temp_city_second = pd.DataFrame()

    fig = px.line_mapbox(us_cities, lat="lat", lon="lon", color="City", zoom=3, height=600)

    for num in range(0, len(optimal_path) - 1):
        temp_city_first = optimal_path[num]
        temp_city_second = optimal_path[num + 1]

        temp_df_first = us_cities.query("city_state == @temp_city_first")
        temp_df_second = us_cities.query("city_state == @temp_city_second")
        
        fig.add_trace(go.Scattermapbox(
            mode="markers+lines",
            lon=[float(temp_df_first["lon"]), float(temp_df_second["lon"])],
            lat=[float(temp_df_first["lat"]), float(temp_df_second["lat"])],
            marker={"size": 10},
            name=num+1
        ))
        
        index += 1

    fig.update_layout(
        mapbox_style="stamen-terrain",
        mapbox_zoom=3,
        mapbox_center_lat = 41,
        margin={"r":0,"t":0,"l":0,"b":0}
    )

    fig.show()

In [8]:
coord = np.array(us_cities[["lat", "lon"]])
dist = result

In [9]:
def sum_dist(tour, dist=dist):
    result = 0
    for i in range(len(tour)-1):  
        result += dist.iloc[tour[i]][dist.columns[tour[i+1]]]

    result += dist.iloc[tour[tour[-1]]][dist.columns[tour[0]]]

    return result

In [44]:
def FindBestPath(coord, max_temp=100000):

    first_tour = []
    first_dist = 0
    first_iter = True

    n = len(coord)
    tour = random.sample(range(n), n)    

    for temp in tqdm(np.linspace(0.0001, 0.0005, max_temp)[::-1]):
        curr_dist = sum_dist(tour)
        
        # generate new tour
        i, j = sorted(random.sample(range(n), 2))    
        new_tour = tour.copy()
        new_tour[i], new_tour[j] = new_tour[j], new_tour[i]
        new_dist = sum_dist(new_tour)

        # save first iteration
        if first_iter:
            first_tour = new_tour.copy()
            first_dist = new_dist
            first_iter = False

        if (np.exp(curr_dist - new_dist) / temp) > random.random():
            tour = new_tour.copy()
    
    print(f'First tour: {first_tour}')
    print(f'First distance: {first_dist}')
    print(f'Best tour: {tour}')
    print(f'Shortest distance: {sum_dist(tour)}')

    return first_tour, first_dist, tour, sum_dist(tour)

In [45]:
f_tour, f_dist, b_tour, b_dist = FindBestPath(coord, 500000)

First tour: [4, 2, 0, 14, 15, 17, 19, 22, 3, 23, 13, 12, 16, 20, 8, 10, 6, 5, 18, 21, 7, 24, 1, 11, 9]
First distance: 42678.775299999994
Best tour: [19, 11, 9, 0, 7, 15, 3, 5, 17, 24, 14, 18, 8, 22, 21, 6, 16, 10, 2, 23, 4, 12, 20, 13, 1]
Shortest distance: 16966.6921


100%|██████████| 500000/500000 [1:12:37<00:00, 114.75it/s]


In [46]:
# path_draw_opt() - is a helper function that drwas the graph way
def path_draw_opt(optimal_path):
    fig = go.Figure()

    index = 0

    temp_city_first = pd.DataFrame()
    temp_city_second = pd.DataFrame()

    fig = px.line_mapbox(us_cities, lat="lat", lon="lon", color="City", zoom=3, height=600)

    for num in range(0, len(optimal_path) - 1):
        temp_city_first = optimal_path[num]
        temp_city_second = optimal_path[num + 1]
        
        fig.add_trace(go.Scattermapbox(
            mode="markers+lines",
            lon=[temp_city_first[1], temp_city_second[1]],
            lat=[temp_city_first[0], temp_city_second[0]],
            marker={"size": 10},
            name=num+1
        ))
        
        index += 1

    fig.add_trace(go.Scattermapbox(
            mode="markers+lines",
            lon=[optimal_path[-1:][0][1], optimal_path[0][1]],
            lat=[optimal_path[-1:][0][0], optimal_path[0][0]],
            marker={"size": 10},
            name=index+1
    ))

    fig.update_layout(
        mapbox_style="stamen-terrain",
        mapbox_zoom=3,
        mapbox_center_lat=41,
        margin={"r":0,"t":0,"l":0,"b":0}
    )

    fig.show()

In [47]:
path_draw_opt(coord[f_tour])
path_draw_opt(coord[b_tour])